# Caluclate value at risk values

In [5]:
import pandas as pd
import os

# Basis path
base_path = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "data", "algorithms_results"))
output_path = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "data", "algorithms_results", "Value_at_Risk"))

#algorithms
algorithm_data = [
    "1_ETC", "2_Greedy", "3_UCB", "4_UCB-Normal", "5_UCB-Tuned", "6_UCB-V", "7_PAC-UCB", "8_UCB-Improved", "9_EUCBV"
]

# Dateikombinationen
combinations = [
    "opt_ver1", "opt_ver2", "opt_ver3",
    "subopt_ver1", "subopt_ver2", "subopt_ver3"
]
alpha_values = [0.01, 0.05, 0.1]

def calculate_value_at_risk(df, alpha):
    timesteps = df['Timestep'].unique()
    return [df[df['Timestep'] == t]['Total Regret'].quantile(1 - alpha) for t in timesteps]

def main():
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for algo in algorithm_data:

        for combination in combinations:
                
            results_path = os.path.join(base_path, f"{algo}_results_{combination}.csv")
            df = pd.read_csv(results_path)
            
            for alpha in alpha_values:
                var_values = calculate_value_at_risk(df, alpha)
                output_file = os.path.join(output_path, f"{algo}_VaR_{combination}_alpha_{alpha}.csv")
                pd.DataFrame({
                    'Timestep': df['Timestep'].unique(),
                    'Value_at_Risk': var_values
                }).to_csv(output_file, index=False)

if __name__ == "__main__":
    main()


## Calculate Value at Risk for Groups of Algorithms as averages

In [ ]:
# Base path
base_path = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "data", "algorithms_results"))

# Variance-aware und nicht-variance-aware Algorithmen
var_aware_algorithms = ["5_UCB-Tuned", "6_UCB-V", "9_EUCBV"]
not_var_aware_algorithms = ["7_PAC-UCB", "8_UCB-Improved"]
standard_algorithms = ["1_ETC", "2_Greedy", "3_UCB", "4_UCB-Normal"]

# Dateikombinationen
combinations = [
    "opt_ver1", "opt_ver2", "opt_ver3",
    "subopt_ver1", "subopt_ver2", "subopt_ver3"
]

# Funktion zum Berechnen des Durchschnitts und Speichern der Ergebnisse
def calculate_average_and_save(algorithms, output_file_suffix):
    for combination in combinations:
        combined_df = None

        for algorithm in algorithms:
            file_path = os.path.join(base_path, f"{algorithm}_results_{combination}.csv")
            df = pd.read_csv(file_path)

            if combined_df is None:
                combined_df = df
            else:
                combined_df.iloc[:, 1:] += df.iloc[:, 1:]  # Addiere die Werte der entsprechenden Spalten

        combined_df.iloc[:, 1:] /= len(algorithms)  # Berechne den Durchschnitt

        output_file_path = os.path.join(base_path, f"{output_file_suffix}_results_{combination}.csv")
        combined_df.to_csv(output_file_path, index=False)

# Berechne und speichere die Durchschnittswerte für variance-aware Algorithmen
calculate_average_and_save(var_aware_algorithms, "Variance-aware UCB Variations")

# Berechne und speichere die Durchschnittswerte für nicht-variance-aware Algorithmen
calculate_average_and_save(not_var_aware_algorithms, "Not-variance-aware UCB Variations")

# Berechne und speichere die Durchschnittswerte für nicht-variance-aware Algorithmen
calculate_average_and_save(standard_algorithms, "Standard Algorithms")

print("Durchschnittswerte erfolgreich berechnet und gespeichert.")
